In [2]:
%pip install python_dotenv
%pip install -U langchain langchain_experimental
%pip install -U spotipy
%pip install -U openai==v0.28.1
%pip install -U gradio==3.48.0
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 72.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 32.4 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.7.1
    Uninstalling gradio_client-0.7.1:
      Successfully uninstalled gradio_client-0.7.1
  Attempting uninstall: gradio
    Found existing installation: gradio 4.8.0
    Uninstalling gradio-4.8.0:
      Successfully uninstalled gradio-4.8.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pydantic

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install -qU \
    pinecone-client \
    pinecone-datasets \
    tiktoken

Note: you may need to restart the kernel to use updated packages.


In [8]:
%reload_ext dotenv
%dotenv

In [4]:
from langchain_experimental.pal_chain import PALChain
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate

In [19]:
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials

In [6]:
import os

In [7]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [25]:
auth = SpotifyOAuth(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"),
    redirect_uri="http://localhost:8888/callback",
    scope="user-library-read user-read-playback-position user-read-recently-played user-top-read",
)

In [26]:
import spotipy
spotifyClient = spotipy.Spotify(auth_manager=auth)
print(spotifyClient.me())

{'display_name': 'Arun', 'external_urls': {'spotify': 'https://open.spotify.com/user/31ajryiecdu4qlhbplqxxtrcyvbu'}, 'href': 'https://api.spotify.com/v1/users/31ajryiecdu4qlhbplqxxtrcyvbu', 'id': '31ajryiecdu4qlhbplqxxtrcyvbu', 'images': [], 'type': 'user', 'uri': 'spotify:user:31ajryiecdu4qlhbplqxxtrcyvbu', 'followers': {'href': None, 'total': 0}}


In [15]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [16]:
from datasets import load_dataset

dataset = load_dataset("maharshipandya/spotify-tracks-dataset")

In [12]:
print(dataset.column_names)

{'train': ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']}


In [17]:
import random

# Function to generate a sentence for danceability
def danceability_sentence(danceability):
    phrases = [
        f"With a danceability score of {danceability}, this track makes you want to get up and move.",
        f"The track has a danceability rating of {danceability}, perfect for dancing.",
        f"You'll find yourself swaying to the rhythm, thanks to its danceability of {danceability}."
    ]
    return random.choice(phrases)

# Function to generate a sentence for energy
def energy_sentence(energy):
    if energy > 0.7:
        return f"It's a high-energy track with a score of {energy}, radiating vibrant vibes."
    elif energy < 0.3:
        return f"With a calm energy level of {energy}, it's a more laid-back track."
    else:
        return f"The track has a balanced energy level of {energy}, neither too calm nor too intense."


In [18]:

# Function for loudness
def loudness_sentence(loudness):
    if loudness > -5:
        return f"With a loudness of {loudness} dB, it's quite a loud track."
    elif loudness < -20:
        return f"It's a softer track with a loudness of {loudness} dB."
    else:
        return f"The track has a moderate loudness level of {loudness} dB."

# Function for mode
def mode_sentence(mode):
    mode_str = "major" if mode == 1 else "minor"
    return f"The track is in the {mode_str} mode."

# Function for speechiness
def speechiness_sentence(speechiness):
    if speechiness > 0.66:
        return "It's predominantly a spoken word track."
    elif 0.33 < speechiness <= 0.66:
        return "The track features a mix of music and speech."
    else:
        return "It's mostly music with minimal speech."

# Function for acousticness
def acousticness_sentence(acousticness):
    if acousticness > 0.5:
        return "This track has a high level of acoustic sound."
    else:
        return "It features more electronic than acoustic sounds."

# Function for instrumentalness
def instrumentalness_sentence(instrumentalness):
    if instrumentalness > 0.5:
        return "It's largely an instrumental track."
    else:
        return "The track includes vocal content."

# Function for liveness
def liveness_sentence(liveness):
    if liveness > 0.8:
        return "There's a high probability the track was recorded live."
    else:
        return "The track likely wasn't recorded in front of a live audience."

# Function for valence
def valence_sentence(valence):
    if valence > 0.5:
        return "It's a track with a more positive, happy vibe."
    else:
        return "The track has a more somber or melancholic tone."

# Function for tempo
def tempo_sentence(tempo):
    return f"The tempo of the track is around {tempo} BPM."

# Function for time signature
def time_signature_sentence(time_signature):
    return f"It has a {time_signature}/4 time signature."

# Incorporating these functions into create_description function
def create_description(example):
    parts = [
        f"Track '{example['track_name']}' by {example['artists']} from the album '{example['album_name']}'",
        danceability_sentence(example['danceability']),
        energy_sentence(example['energy']),
        loudness_sentence(example['loudness']),
        mode_sentence(example['mode']),
        speechiness_sentence(example['speechiness']),
        acousticness_sentence(example['acousticness']),
        instrumentalness_sentence(example['instrumentalness']),
        liveness_sentence(example['liveness']),
        valence_sentence(example['valence']),
        tempo_sentence(example['tempo']),
        time_signature_sentence(example['time_signature']),
        f"The genre is categorized as {example['track_genre']}.",
        f"It has a popularity score of {example['popularity']} and lasts for {example['duration_ms'] / 1000} seconds."
    ]
    example['textual_description'] = ' '.join(parts)
    return example


In [19]:
dataset = dataset.map(create_description)

In [20]:
print(dataset.column_names)
print(dataset["train"][0])

{'train': ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre', 'textual_description']}
{'Unnamed: 0': 0, 'track_id': '5SuOikwiRyPMVoIQDJUgSV', 'artists': 'Gen Hoshino', 'album_name': 'Comedy', 'track_name': 'Comedy', 'popularity': 73, 'duration_ms': 230666, 'explicit': False, 'danceability': 0.676, 'energy': 0.461, 'key': 1, 'loudness': -6.746, 'mode': 0, 'speechiness': 0.143, 'acousticness': 0.0322, 'instrumentalness': 1.01e-06, 'liveness': 0.358, 'valence': 0.715, 'tempo': 87.917, 'time_signature': 4, 'track_genre': 'acoustic', 'textual_description': "Track 'Comedy' by Gen Hoshino from the album 'Comedy' With a danceability score of 0.676, this track makes you want to get up and move. The track has a balanced energy level of 0.461, neither too calm nor too intense.

In [43]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import pinecone
import numpy as np

small_dataset_to_vectorize = dataset['train'].select(range(10000))

# Initialize the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Pinecone setup
pinecone.init(api_key='ef418cba-180e-498f-be6a-4c38711964d2', environment='gcp-starter')
index_name = 'llm-music-chatbot'

# Create the Pinecone index if it does not exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=384)

# Connect to the Pinecone index
index = pinecone.Index(index_name)

# Function to vectorize and upload to Pinecone
def vectorize_and_upload(batch):
    # Vectorize the textual descriptions in the batch
    embeddings = model.encode(batch['textual_description'])
    
    # Convert embeddings to lists
    embeddings_list = embeddings.tolist()

    # Create a list of tuples for uploading
    batch_for_upload = []
    for track_id, emb in zip(batch['track_id'], embeddings_list):
        # Ensure track_id is a string and add it to the batch
        batch_for_upload.append((str(track_id), emb))

    # Upload the batch to Pinecone
    index.upsert(vectors=batch_for_upload)

# Apply the function to the dataset
dataset.map(vectorize_and_upload, batched=True, batch_size=1000)

# from math import ceil

# total_rows = 114000
# chunk_size = 10000  # Process 10k rows at a time
# batch_size = 160    # Batch size for upsert

# # Calculate the number of chunks
# num_chunks = ceil(total_rows / chunk_size)

# for chunk in range(num_chunks):
#     start_row = chunk * chunk_size
#     end_row = min((chunk + 1) * chunk_size, total_rows)

#     # Slice the dataset for the current chunk
#     current_chunk = dataset["train"].select(range(start_row, end_row))

#     # Process and upsert the current chunk in batches
#     current_chunk.map(vectorize_and_upload, batched=True, batch_size=batch_size)

# Close the index connection when done
# index.close()  # Uncomment to close the Pinecone index connection


Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x1157f7240>
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):

KeyboardInterrupt: 


NameError: name 'dataset' is not defined

In [44]:
def query_pinecone(query):
    query_vector = model.encode(query).tolist()
    results = index.query(vector=query_vector, top_k=5)
    return results

In [45]:
query_pinecone("I want to listen to some beatles music that's rock")

NameError: name 'model' is not defined

In [82]:
recently_played = spotifyClient.current_user_recently_played(limit=2)
print(recently_played)
for track in recently_played['items']:
    print(track['track']['name'], "-", track['track']['artists'][0]['name'])

{'items': [{'track': {'album': {'album_type': 'compilation', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'}, 'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of', 'id': '0LyfQWJT6nXafLPZqxe9Of', 'name': 'Various Artists', 'type': 'artist', 'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'B

In [83]:
# Top Tracks
top_tracks = spotifyClient.current_user_top_tracks(limit=2)
print(top_tracks)
for track in top_tracks['items']:
    print(track['name'], "-", track['artists'][0]['name'])

# Top Artists
top_artists = spotifyClient.current_user_top_artists(limit=2)
print(top_artists)
for artist in top_artists['items']:
    print(artist['name'])

{'items': [{'album': {'album_type': 'SINGLE', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6AiX12wXdXFoGJ2vk8zBjy'}, 'href': 'https://api.spotify.com/v1/artists/6AiX12wXdXFoGJ2vk8zBjy', 'id': '6AiX12wXdXFoGJ2vk8zBjy', 'name': 'Yuvan Shankar Raja', 'type': 'artist', 'uri': 'spotify:artist:6AiX12wXdXFoGJ2vk8zBjy'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/7yv0wfPOLwqdb7HdTeIi4Q'}, 'href': 'https://api.spotify.com/v1/albums/7yv0wfPOLwqdb7HdTeIi4Q', 'i

In [84]:
saved_tracks = spotifyClient.current_user_saved_tracks(limit=2)
print(saved_tracks)
for item in saved_tracks['items']:
    track = item['track']
    print(track['name'], "-", track['artists'][0]['name'])

{'href': 'https://api.spotify.com/v1/me/tracks?offset=0&limit=2', 'items': [{'added_at': '2023-11-03T02:16:33Z', 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0fhunuOz2OuOEr3YK5ctSm'}, 'href': 'https://api.spotify.com/v1/artists/0fhunuOz2OuOEr3YK5ctSm', 'id': '0fhunuOz2OuOEr3YK5ctSm', 'name': 'Thalapathy Vijay', 'type': 'artist', 'uri': 'spotify:artist:0fhunuOz2OuOEr3YK5ctSm'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/3JGrxsZ6kfzJrfwsxwERhS'}, 'href': 'https://api.spotify.com/v1/artists/3JGrxsZ6kfzJrfwsxwERhS', 'id': '3JGrxsZ6kfzJrfwsxwERhS', 'name': 'M.M.Manasi', 'type': 'artist', 'uri': 'spotify:artist:3JGrxsZ6kfzJrfwsxwERhS'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/2FgHPfRprDaylrSRVf1UlN'}, 'href': 'https://api.spotify.com/v1/artists/2FgHPfRprDaylrSRVf1UlN', 'id': '2FgHPfRprDaylrSRVf1UlN', 'name': 'Thaman S', 'type': 'artist', 'uri': 'spotify:artist:2FgHPfRprDaylrSR

In [85]:
playlists = spotifyClient.current_user_playlists(limit=2)
print(playlists)
for playlist in playlists['items']:
    print(playlist['name'])

{'href': 'https://api.spotify.com/v1/users/31ajryiecdu4qlhbplqxxtrcyvbu/playlists?offset=0&limit=2', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/7bm5HoaNIFFd37wChWDp8j'}, 'href': 'https://api.spotify.com/v1/playlists/7bm5HoaNIFFd37wChWDp8j', 'id': '7bm5HoaNIFFd37wChWDp8j', 'images': [{'height': 640, 'url': 'https://mosaic.scdn.co/640/ab67616d0000b2731c685496e306c633835c96beab67616d0000b2735675e83f707f1d7271e5cf8aab67616d0000b273813713582dcc508e7d5073c4ab67616d0000b273c6f7af36ecdc3ed6e0a1f169', 'width': 640}, {'height': 300, 'url': 'https://mosaic.scdn.co/300/ab67616d0000b2731c685496e306c633835c96beab67616d0000b2735675e83f707f1d7271e5cf8aab67616d0000b273813713582dcc508e7d5073c4ab67616d0000b273c6f7af36ecdc3ed6e0a1f169', 'width': 300}, {'height': 60, 'url': 'https://mosaic.scdn.co/60/ab67616d0000b2731c685496e306c633835c96beab67616d0000b2735675e83f707f1d7271e5cf8aab67616d0000b273813713582dcc508e7d5073c4ab67616d0000b2

In [86]:
track_ids = [track['track']['id'] for track in recently_played['items']]

# Fetching audio features for the tracks
audio_features = spotifyClient.audio_features(track_ids)
print(audio_features)

for feature in audio_features:
    if feature:  # Check if feature data is available
        print(f"Audio Features for Track ID: {feature['id']}")
        for key, value in feature.items():
            print(f" - {key.capitalize().replace('_', ' ')}: {value}")
        print()

[{'danceability': 0.694, 'energy': 0.479, 'key': 0, 'loudness': -8.918, 'mode': 1, 'speechiness': 0.0307, 'acousticness': 0.36, 'instrumentalness': 0, 'liveness': 0.195, 'valence': 0.735, 'tempo': 143.974, 'type': 'audio_features', 'id': '01yHYYvFwxQorNJTBwbvPn', 'uri': 'spotify:track:01yHYYvFwxQorNJTBwbvPn', 'track_href': 'https://api.spotify.com/v1/tracks/01yHYYvFwxQorNJTBwbvPn', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/01yHYYvFwxQorNJTBwbvPn', 'duration_ms': 369774, 'time_signature': 4}, {'danceability': 0.656, 'energy': 0.737, 'key': 0, 'loudness': -8.575, 'mode': 1, 'speechiness': 0.0274, 'acousticness': 0.218, 'instrumentalness': 0.000545, 'liveness': 0.0838, 'valence': 0.602, 'tempo': 127.044, 'type': 'audio_features', 'id': '4eokh1sHDGf02i2nGagwrn', 'uri': 'spotify:track:4eokh1sHDGf02i2nGagwrn', 'track_href': 'https://api.spotify.com/v1/tracks/4eokh1sHDGf02i2nGagwrn', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4eokh1sHDGf02i2nGagwrn', 'duration

In [20]:
# Fetch the top 10 artists
top_artists = spotifyClient.current_user_top_artists(limit=10)
for artist in top_artists['items']:
    artist_details = spotifyClient.artist(artist['id'])
    print(f"Artist: {artist_details['name']}")
    print(f"Genres: {', '.join(artist_details['genres'])}")
    print(f"Popularity: {artist_details['popularity']}")
    print()

# Fetch the top 10 tracks
top_tracks = spotifyClient.current_user_top_tracks(limit=10)
album_ids = set()  # To avoid duplicate albums
for track in top_tracks['items']:
    album_id = track['album']['id']
    if album_id not in album_ids:
        album_ids.add(album_id)
        album_details = spotifyClient.album(album_id)
        print(f"Album: {album_details['name']}")
        print(f"Release Date: {album_details['release_date']}")
        print(f"Total Tracks: {album_details['total_tracks']}")
        print()

Artist: Anirudh Ravichander
Genres: desi hip hop, filmi, tamil hip hop
Popularity: 84

Artist: A.R. Rahman
Genres: filmi
Popularity: 77

Artist: Yuvan Shankar Raja
Genres: filmi, kollywood
Popularity: 73

Artist: Harris Jayaraj
Genres: filmi
Popularity: 70

Artist: G. V. Prakash
Genres: filmi, kollywood
Popularity: 70

Artist: Thaman S
Genres: new beat, tamil pop
Popularity: 68

Artist: Karthik
Genres: filmi
Popularity: 71

Artist: S. P. Balasubrahmanyam
Genres: filmi
Popularity: 75

Artist: Devi Sri Prasad
Genres: tollywood
Popularity: 66

Artist: Hariharan
Genres: filmi, ghazal
Popularity: 71

Album: Deepavali (Original Motion Picture Soundtrack)
Release Date: 2007-01-05
Total Tracks: 5

Album: Chaleya (From "Jawan")
Release Date: 2023-08-14
Total Tracks: 1

Album: Hukum - Thalaivar Alappara (From "Jailer")
Release Date: 2023-07-17
Total Tracks: 1

Album: Alaipayuthey
Release Date: 2000-12-31
Total Tracks: 10

Album: Osarattum Pathu Thala (From "Pathu Thala")
Release Date: 2023-03-28

In [22]:
import pinecone
from sentence_transformers import SentenceTransformer
import json

In [53]:
pinecone.init(api_key="ef418cba-180e-498f-be6a-4c38711964d2", environment='gcp-starter')
index_name = "llm-music-chatbot"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=384, metric="cosine")  # Adjust dimension as needed
index = pinecone.Index(index_name)

In [74]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [41]:
# Example of vectorizing a piece of text
def vectorize(text):
    return model.encode(text).tolist()

In [44]:
# Example of adding a vector to Pinecone
def upload_to_pinecone(data_strings, category):
    for i, data_string in enumerate(data_strings):
        vector = vectorize(data_string)
        id = f"{category}-{i}"
        index.upsert(vectors={id: vector})

In [68]:
def query_pinecone(query):
    query_vector = vectorize(query)
    results = index.query(vector=query_vector, top_k=5)
    return results

In [51]:
recently_played_strings = []
recently_played_track_ids = []
for item in recently_played['items']:
    track = item['track']
    track_string = f"Recently played track: {track['name']} by {track['artists'][0]['name']} from the album {track['album']['name']} on {item['played_at']}"
    recently_played_strings.append(track_string)
    recently_played_track_ids.append(track['id'])  # Add the track ID to the list
print(recently_played_strings)
print(recently_played_track_ids)

['Recently played track: Sodakku by Anirudh Ravichander from the album Thaanaa Serndha Koottam (Original Motion Picture Soundtrack) on 2023-12-01T10:20:28.928Z', 'Recently played track: Uyirin Uyire by KK from the album Kaakha Kaakha (Original Motion Picture Soundtrack) on 2023-12-01T10:04:44.246Z', 'Recently played track: Ordinary Person (From "Leo") by Anirudh Ravichander from the album Ordinary Person (From "Leo") on 2023-12-01T03:33:42.088Z', 'Recently played track: Naa Ready (From "Leo") by Anirudh Ravichander from the album Naa Ready (From "Leo") on 2023-12-01T03:31:23.140Z', 'Recently played track: Naa Ready (From "Leo") by Anirudh Ravichander from the album Naa Ready (From "Leo") on 2023-12-01T02:54:08.400Z', 'Recently played track: I\'m Scared (From "Leo") by Anirudh Ravichander from the album I\'m Scared (From "Leo") on 2023-11-30T21:56:02.002Z', 'Recently played track: Badass (From "Leo") by Anirudh Ravichander from the album Badass (From "Leo") on 2023-11-30T21:53:28.119Z',

In [56]:
top_tracks_strings = []
top_tracks_track_ids = []
for track in top_tracks['items']:
    track_string = f"Top track: {track['name']} by {track['artists'][0]['name']} from the album {track['album']['name']}"
    top_tracks_strings.append(track_string)
    top_tracks_track_ids.append(track['id'])  # Add the track ID to the list
print(top_tracks_strings)
print(top_tracks_track_ids)

['Top track: Kaadhal Vaithu by Yuvan Shankar Raja from the album Deepavali (Original Motion Picture Soundtrack)', 'Top track: Chaleya (From "Jawan") by Anirudh Ravichander from the album Chaleya (From "Jawan")', 'Top track: Hukum - Thalaivar Alappara (From "Jailer") by Anirudh Ravichander from the album Hukum - Thalaivar Alappara (From "Jailer")', 'Top track: Endrendrum Punnagai by Srinivas from the album Alaipayuthey', 'Top track: Osarattum Pathu Thala (From "Pathu Thala") by A.R. Rahman from the album Osarattum Pathu Thala (From "Pathu Thala")', 'Top track: Believer by Imagine Dragons from the album Evolve', 'Top track: Pookkalae Sattru Oyivedungal by A.R. Rahman from the album I (Original Motion Picture Soundtrack)', 'Top track: Dhimu Dhimu by Harris Jayaraj from the album Engeyum Kadhal (Original Motion Picture Soundtrack)', 'Top track: Pogadhe by Yuvan Shankar Raja from the album Deepavali (Original Motion Picture Soundtrack)', 'Top track: Challa by A.R. Rahman from the album Jab 

In [58]:
saved_tracks_strings = []
saved_tracks_track_ids = []
for item in saved_tracks['items']:
    track = item['track']
    track_string = f"Saved track: {track['name']} by {track['artists'][0]['name']} from the album {track['album']['name']}"
    saved_tracks_strings.append(track_string)
    saved_tracks_track_ids.append(track['id'])  # Add the track ID to the list
print(saved_tracks_strings)
print(saved_tracks_track_ids)

['Saved track: Ranjithame (From "Varisu") by Thalapathy Vijay from the album Ranjithame (From "Varisu")', 'Saved track: Ennai Konjum by Tippu from the album Kaakha Kaakha (Original Motion Picture Soundtrack)', 'Saved track: Kaadhal Mazhaiye by Srinivas from the album Jay Jay (Original Motion Picture Soundtrack)', 'Saved track: Yathe Yathe by G. V. Prakash from the album Aadukalam (Original Motion Picture Soundtrack)', 'Saved track: Challa by A.R. Rahman from the album Jab Tak Hai Jaan', 'Saved track: The Life of Ram by Govind Vasantha from the album 96 (Original Motion Picture Soundtrack)', 'Saved track: Dil Se Re by A.R. Rahman from the album Dil Se (Original Motion Picture Soundtrack)', 'Saved track: Naakka Mukka by Vijay Antony from the album Kadhalil Vizhundhen', 'Saved track: Thai Mannai Vanakkam - Tamil Version of Maa Tujhe Salaam by A.R. Rahman from the album Vande Mataram', 'Saved track: Maa Tujhe Salaam by A.R. Rahman from the album Vande Mataram', 'Saved track: Chaleya (From 

In [59]:
top_artists_strings = []
top_artists_ids = []
for artist in top_artists['items']:
    artist_string = f"Top artist: {artist['name']} known for {', '.join(artist['genres'])} with popularity score of {artist['popularity']}"
    top_artists_strings.append(artist_string)
    top_artists_ids.append(artist['id'])  # Add the artist ID to the list
print(top_artists_strings)
print(top_artists_ids)

['Top artist: Anirudh Ravichander known for desi hip hop, filmi, tamil hip hop with popularity score of 84', 'Top artist: A.R. Rahman known for filmi with popularity score of 77', 'Top artist: Yuvan Shankar Raja known for filmi, kollywood with popularity score of 73', 'Top artist: Harris Jayaraj known for filmi with popularity score of 70', 'Top artist: G. V. Prakash known for filmi, kollywood with popularity score of 70', 'Top artist: Thaman S known for new beat, tamil pop with popularity score of 68', 'Top artist: Karthik known for filmi with popularity score of 71', 'Top artist: S. P. Balasubrahmanyam known for filmi with popularity score of 75', 'Top artist: Devi Sri Prasad known for tollywood with popularity score of 66', 'Top artist: Hariharan known for filmi, ghazal with popularity score of 71']
['4zCH9qm4R2DADamUHMCa6O', '1mYsTxnqsietFxj1OgoGbG', '6AiX12wXdXFoGJ2vk8zBjy', '29aw5YCdIw2FEXYyAJZI8l', '5VVN3xZw1i2qihfITZlvCZ', '2FgHPfRprDaylrSRVf1UlN', '0LSPREIgGMZXCuKVel7LVD', '2a

In [60]:
audio_features_strings = []
audio_ids = []
for feature in audio_features:
    features_list = [f"{key}: {value}" for key, value in feature.items() if key != 'type' and key != 'uri' and key != 'track_href' and key != 'analysis_url']
    feature_string = f"Audio features for Track ID {feature['id']}: {', '.join(features_list)}"
    audio_features_strings.append(feature_string)
    audio_ids.append(feature['id'])  # Add the track ID to the list
print(audio_features_strings)
print(audio_ids)

['Audio features for Track ID 1yotVCnfaPDvp9C7LsdGB1: danceability: 0.741, energy: 0.9, key: 11, loudness: -3.634, mode: 1, speechiness: 0.27, acousticness: 0.344, instrumentalness: 0.0039, liveness: 0.0362, valence: 0.663, tempo: 159.557, id: 1yotVCnfaPDvp9C7LsdGB1, duration_ms: 238913, time_signature: 4', 'Audio features for Track ID 7LuYc1yBvfvvLC1O7Ng4v4: danceability: 0.689, energy: 0.805, key: 4, loudness: -8.496, mode: 0, speechiness: 0.058, acousticness: 0.485, instrumentalness: 1.46e-06, liveness: 0.142, valence: 0.834, tempo: 135.528, id: 7LuYc1yBvfvvLC1O7Ng4v4, duration_ms: 321427, time_signature: 4', 'Audio features for Track ID 6FahmzZYKH0zb2f9hrVsvw: danceability: 0.724, energy: 0.851, key: 4, loudness: -6.068, mode: 0, speechiness: 0.0297, acousticness: 0.00196, instrumentalness: 0.038, liveness: 0.0513, valence: 0.81, tempo: 99.998, id: 6FahmzZYKH0zb2f9hrVsvw, duration_ms: 139200, time_signature: 4', 'Audio features for Track ID 1B02UI29t3PTh3m98absaP: danceability: 0.7

In [61]:
playlists_strings = []
playlists_ids = []
for playlist in playlists['items']:
    playlist_string = f"Playlist: {playlist['name']}, Description: {playlist['description']}, Total Tracks: {playlist['tracks']['total']}"
    playlists_strings.append(playlist_string)
    playlists_ids.append(playlist['id'])  # Add the playlist ID to the list
print(playlists_strings)
print(playlists_ids)

['Playlist: Ishana’s favourites, Description: , Total Tracks: 5']
['7bm5HoaNIFFd37wChWDp8j']


In [55]:
# Modified function to work with a list of pre-formatted strings and a separate list of IDs
def upload_tracks_to_pinecone(track_strings, track_ids, category):
    batch = []
    for track_string, track_id in zip(track_strings, track_ids):
        vector = vectorize(track_string)
        pinecone_id = f"{category}-{track_id}"  # Create a unique identifier
        batch.append((pinecone_id, vector))
    
    # Upsert in batch
    index.upsert(vectors=batch)

# Example usage
upload_tracks_to_pinecone(recently_played_strings, recently_played_track_ids, 'recently_played')


In [57]:
# upload top tracks to pinecone
upload_tracks_to_pinecone(top_tracks_strings, top_tracks_track_ids, 'top_tracks')


In [62]:
upload_tracks_to_pinecone(saved_tracks_strings, saved_tracks_track_ids, 'saved_tracks')


In [63]:
upload_tracks_to_pinecone(audio_features_strings, audio_ids, 'audio_features')  # Reusing top_tracks_ids for simplicity


In [64]:
def upload_artists_to_pinecone(artist_strings, artist_ids, category):
    batch = []
    for artist_string, artist_id in zip(artist_strings, artist_ids):
        vector = vectorize(artist_string)
        pinecone_id = f"{category}-{artist_id}"  # Create a unique identifier
        batch.append((pinecone_id, vector))
    
    # Upsert in batch
    index.upsert(vectors=batch)

# Example usage (assuming you have top_artists_strings and top_artists_ids lists ready)
upload_artists_to_pinecone(top_artists_strings, top_artists_ids, 'top_artists')


In [65]:
def upload_playlists_to_pinecone(playlist_strings, playlist_ids, category):
    batch = []
    for playlist_string, playlist_id in zip(playlist_strings, playlist_ids):
        vector = vectorize(playlist_string)
        pinecone_id = f"{category}-{playlist_id}"  # Create a unique identifier
        batch.append((pinecone_id, vector))
    
    # Upsert in batch
    index.upsert(vectors=batch)

# Example usage (assuming you have playlists_strings and playlists_ids lists ready)
upload_playlists_to_pinecone(playlists_strings, playlists_ids, 'playlists')


In [72]:
query_pinecone("can you recommend some songs with high beat for gym workout?")

{'matches': [{'id': 'audio_features-1B02UI29t3PTh3m98absaP',
              'score': 0.417991817,
              'values': []},
             {'id': 'audio_features-09Q0LYcLol1C8HOZ6pIRdO',
              'score': 0.41640538,
              'values': []},
             {'id': 'audio_features-6EJ1ki4Dcf6in9eidtPSNZ',
              'score': 0.408694059,
              'values': []},
             {'id': 'audio_features-7BOMm2gouxtIB2oxfSzPi6',
              'score': 0.404787213,
              'values': []},
             {'id': 'audio_features-6FahmzZYKH0zb2f9hrVsvw',
              'score': 0.402909458,
              'values': []}],
 'namespace': ''}

In [27]:
SPOTIPY_PROMPT_TEMPLATE = (
    '''
API LIMITATIONS TO NOTE
* When requesting track information, the limit is 50 at a time
* When requesting audio features, the limit is 100 at a time
* When selecting multiple artists, the limit is 50 at a time
* When asking for recommendations, the limit is 50 at a time
=====

Q: What albums has the band Green Day made?

# solution in Python:


def solution():
    """What albums has the band Green Day made?"""
    search_results = sp.search(q='Green Day', type='artist')
    uri = search_results['artists']['items'][0]['uri']
    albums = sp.artist_albums(green_day_uri, album_type='album')
    return albums




Q: Who are some musicians similar to Fiona Apple?

# solution in Python:


def solution():
    """Who are some musicians similar to Fiona Apple?"""
    search_results = sp.search(q='Fiona Apple', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    artists = sp.artist_related_artists(uri)
    return artists



Q: Tell me what songs by The Promise Ring sound like

# solution in Python:


def solution():
    """Tell me what songs by The Promise Ring sound like?"""
    search_results = sp.search(q='The Promise Ring', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    tracks = sp.artist_top_tracks(uri)
    track_uris = [track.get('uri') for track in tracks['tracks']]
    audio_details = sp.audio_features(track_uris)
    return audio_details



Q: Get me the URI for the album The Colour And The Shape

# solution in Python:


def solution():
    """Get me the URI for the album The Colour And The Shape"""
    search_results = sp.search(q='The Colour And The Shape', type='album')
    uri = search_results['albums']['items'][0].get('uri')
    return uri



Q: What are the first three songs on Diet Cig's Over Easy?

# solution in Python:


def solution():
    """What are the first three songs on Diet Cig's Over Easy?"""
    # Get the URI for the album
    search_results = sp.search(q='Diet Cig Over Easy', type='album')
    album = search_results['albums']['items'][0]
    album_uri = album['uri']
    # Get the album tracks
    album_tracks = sp.album_tracks(album_uri)['items']
    # Sort the tracks by duration
    first_three = album_tracks[:3]
    tracks = []
    # Only include relevant fields
    for i, track in enumerate(first_three):
        # track['album'] does NOT work with sp.album_tracks
        # you need to use album['name'] instead
        tracks.append({{
            'position': i+1,
            'song_name': track.get('name'),
            'song_uri': track['artists'][0].get('uri'),
            'artist_uri': track['artists'][0].get('uri'),
            'album_uri': album.get('uri'),
            'album_name': album.get('name')
        }})
    return tracks


Q: What are the thirty most danceable songs by Metallica?

# solution in Python:


def solution():
    """What are most danceable songs by Metallica?"""
    search_results = sp.search(q='Metallica', type='artist')
    uri = search_results['artists']['items'][0]['uri']
    albums = sp.artist_albums(uri, album_type='album')
    album_uris = [album['uri'] for album in albums['items']]
    tracks = []
    for album_uri in album_uris:
        album_tracks = sp.album_tracks(album_uri)
        tracks.extend(album_tracks['items'])
    track_uris = [track['uri'] for track in tracks]
    danceable_tracks = []
    # You can only have 100 at a time
    for i in range(0, len(track_uris), 100):
        subset_track_uris = track_uris[i:i+100]
        audio_details = sp.audio_features(subset_track_uris)
        for j, details in enumerate(audio_details):
            if details['danceability'] > 0.7:
                track = tracks[i+j]
                danceable_tracks.append({{
                    'song': track.get('name')
                    'album': track.get('album').get('name')
                    'danceability': details.get('danceability'),
                    'tempo': details.get('tempo'),
                }})
                # Be sure to add the audio details to the track
                danceable_tracks.append(track)
    return danceable_tracks



Q: {question}. Return a list or dictionary, including all the fields, including relevant scores and the uris to the albums/songs/artists mentioned. Only return the data – if the prompt asks for a format such as markdown or a simple string, ignore it: you are only meant to provide the information, not the formatting. A later step in the process will convert the data into the new format (table, sentence, etc).

# solution in Python:
'''.strip()
    + "\n\n\n"
)

SPOTIPY_PROMPT = PromptTemplate(input_variables=["question"], template=SPOTIPY_PROMPT_TEMPLATE)

In [28]:
spotify_chain = PALChain(
    llm_chain=LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo"), prompt=SPOTIPY_PROMPT),
    get_answer_expr="import json; print(json.dumps(solution()))",
    python_globals={"sp": spotifyClient},
    stop="\n\n\n",
    verbose=True,
    return_intermediate_steps=True,
)
print(spotify_chain)

verbose=True llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='API LIMITATIONS TO NOTE\n* When requesting track information, the limit is 50 at a time\n* When requesting audio features, the limit is 100 at a time\n* When selecting multiple artists, the limit is 50 at a time\n* When asking for recommendations, the limit is 50 at a time\n=====\n\nQ: What albums has the band Green Day made?\n\n# solution in Python:\n\n\ndef solution():\n    """What albums has the band Green Day made?"""\n    search_results = sp.search(q=\'Green Day\', type=\'artist\')\n    uri = search_results[\'artists\'][\'items\'][0][\'uri\']\n    albums = sp.artist_albums(green_day_uri, album_type=\'album\')\n    return albums\n\n\n\n\nQ: Who are some musicians similar to Fiona Apple?\n\n# solution in Python:\n\n\ndef solution():\n    """Who are some musicians similar to Fiona Apple?"""\n    search_results = sp.search(q=\'Fiona Apple\', type=\'artist\')\n    uri = search_results[\'artist

In [29]:
PROMPT_TEMPLATE_TO_CLEANUP_RESPONSE = (""" 
Using this code:

```python
{intermediate_steps}
```

We got the following output from the Spotify API:

```json
{result}
```

Using the output above as your data source, answer the question {question}. Don't describe the code or process, just answer the question.
Answer:"""
)

PROMPT_RESPONSE_CLEANUP = PromptTemplate(
    input_variables=["question", "intermediate_steps", "result"],
    template=PROMPT_TEMPLATE_TO_CLEANUP_RESPONSE,
)

In [30]:
explainer_chain = LLMChain(
    llm=llm,
    prompt=PROMPT_RESPONSE_CLEANUP,
    verbose=True,
    output_key='answer'
)

In [31]:
overall_chain = SequentialChain(
    chains=[spotify_chain, explainer_chain],
    input_variables=['question'],
    verbose=True
)

In [37]:
print(overall_chain.run("can you recommend some happy songs from the hollywood"))



> Entering new SequentialChain chain...


> Entering new PALChain chain...
def solution():
    """Recommend some happy songs from the hollywood."""
    search_results = sp.search(q='Hollywood', type='track', market='US', limit=50)
    tracks = search_results['tracks']['items']
    happy_songs = []
    for track in tracks:
        audio_details = sp.audio_features(track['uri'])
        if audio_details[0]['valence'] > 0.7:
            happy_songs.append({
                'song_name': track['name'],
                'artist_name': track['artists'][0]['name'],
                'album_name': track['album']['name'],
                'song_uri': track['uri'],
                'artist_uri': track['artists'][0]['uri'],
                'album_uri': track['album']['uri'],
                'valence_score': audio_details[0]['valence']
            })
    return happy_songs

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
 
Using this code:

```python
def solution():
    "

In [38]:
import gradio as gr
import random
import time


In [41]:
import gradio as gr
import random
import time

def respond(message, chat_history):
    # You can now use the name and age in your response logic if needed
    bot_message = overall_chain.run(message)
    chat_history.append((f"{message}", bot_message))
    time.sleep(2)
    return "", chat_history

with gr.Blocks() as demo:

    with gr.Row():
        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="Your Message")
        clear = gr.ClearButton([msg, chatbot])

    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])

demo.launch()


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
 
Using this code:

```python
def solution():
    """Recommend some happy songs from Bollywood"""
    search_results = sp.search(q='Bollywood', type='playlist')
    playlist_uri = search_results['playlists']['items'][0]['uri']
    tracks = sp.playlist_tracks(playlist_uri)['items']
    happy_songs = []
    for track in tracks:
        audio_details = sp.audio_features(track['track']['uri'])
        if audio_details[0]['valence'] > 0.7:
            song = track['track']['name']
            album = track['track']['album']['name']
            artist = track['track']['artists'][0]['name']
            happy_songs.append({
                'song': song,
                'album': album,
                'artist': artist,
                'valence': audio_details[0]['valence'],
                'energy': audio_details[0]['energy']
            })
    return happy_songs
```

We got the following output from the Spotify API: